In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import sys
!{sys.executable} -m pip install requests
import requests

In [5]:
#let's load our iMDB dataset using pd.read_csv
api_key = "fef79dcd";
movies = pd.read_csv("imdb.csv")
movies.head()


,fn,tid,title,wordsInTitle,url,imdbRating,ratingCount,duration,year,type,...,Short,Sport,TalkShow,Thriller,War,Western,asdf,untitled,Unnamed: 46,Unnamed: 47
0,titles01/tt0012349,tt0012349,Der Vagabund und das Kind (1921),der vagabund und das kind,http://www.imdb.com/title/tt0012349/,8.4,40550,3240,1921,video.movie,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
1,titles01/tt0015864,tt0015864,Goldrausch (1925),goldrausch,http://www.imdb.com/title/tt0015864/,8.3,45319,5700,1925,video.movie,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
2,titles01/tt0017136,tt0017136,Metropolis (1927),metropolis,http://www.imdb.com/title/tt0017136/,8.4,81007,9180,1927,video.movie,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
3,titles01/tt0017925,tt0017925,Der General (1926),der general,http://www.imdb.com/title/tt0017925/,8.3,37521,6420,1926,video.movie,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
4,titles01/tt0021749,tt0021749,Lichter der Großstadt (1931),lichter der gro stadt,http://www.imdb.com/title/tt0021749/,8.7,70057,5220,1931,video.movie,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN


In [6]:
#we can see some movies with strange titles, such as this one, have their titles spill into other fields
movies.iloc[[64]]

,fn,tid,title,wordsInTitle,url,imdbRating,ratingCount,duration,year,type,...,Short,Sport,TalkShow,Thriller,War,Western,asdf,untitled,Unnamed: 46,Unnamed: 47
64,titles01/tt0057012,tt0057012,Dr. Seltsam\,oder wie ich lernte\,die Bombe zu lieben (1964),dr seltsam oder wie ich lernte die bombe zu li...,http://www.imdb.com/title/tt0057012/,8.6,263157,5700,...,0,0,0,0,0,0,1.0,0.0,NaN,NaN


In [7]:
#removing all the messed up rows with overflowing titles
movies["imdbRating"][0]
movies_1 = pd.to_numeric(movies["imdbRating"], errors = "coerce")
movies = (movies[movies_1.notnull()])
# print(len(movies))
# print(movies)


In [8]:
#Now that row, and others like it, are deleted
movies.iloc[[64]]

,fn,tid,title,wordsInTitle,url,imdbRating,ratingCount,duration,year,type,...,Short,Sport,TalkShow,Thriller,War,Western,asdf,untitled,Unnamed: 46,Unnamed: 47
65,titles01/tt0057115,tt0057115,Gesprengte Ketten (1963),gesprengte ketten,http://www.imdb.com/title/tt0057115/,8.3,120161,10320,1963,video.movie,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN


In [9]:
rt_ratings = []
box_office = []

In [10]:
#generate the rotten tomatoes scores and box office for the first 1000 movies

for i in range(1000):
    row = movies.iloc[i];
    movie_id = row["tid"];
    response =  requests.get("http://www.omdbapi.com/?i="+movie_id+"&apikey="+api_key+"&");
    r_json = response.json()
    try:
        box_office.append(r_json["BoxOffice"])
    except:
        box_office.append("NaN")
    try:
        rt_ratings.append(r_json["Ratings"][1]["Value"])
    except:
        rt_ratings.append("NaN")



In [11]:
#Let's check out what that did for us
print(len(rt_ratings))
print(rt_ratings)


1000
['100%', '100%', '97%', '93%', '98%', '100%', '98%', '100%', '91%', '85%', '98%', '100%', '93%', '100%', '100%', '100%', '99%', '97%', '96%', '97%', '94%', '98%', '98%', '94%', '100%', '99%', '99%', '98%', '98%', '98%', '96%', '100%', '100%', '98%', '97%', '96%', '89%', '99%', '99%', '100%', '97%', '99%', '98%', '100%', '95%', '95%', '93%', '95%', '100%', '96%', '95%', '86%', '99%', '100%', '95%', '93%', '96%', '96%', '98%', '91%', '95%', '98%', '92%', '98%', '94%', '98%', '99%', '91%', '97%', '90%', '95%', '100%', '86%', '93%', '90%', '95%', '87%', '98%', '94%', '99%', '97%', '97%', '90%', '95%', '98%', '93%', '92%', '94%', '95%', '98%', '92%', '93%', '98%', '98%', '95%', '100%', '92%', '94%', '94%', '85%', '98%', '95%', '90%', '100%', '85%', '84%', '89%', '82%', '93%', '88%', '86%', '100%', '96%', '97%', '97%', '88%', '91%', '96%', '91%', '97%', '93%', '98%', '91%', '94%', '88%', '96%', '94%', '96%', '93%', '92%', '96%', '96%', '94%', '97%', '70%', '93%', '73%', '91%', '90%', '1

In [12]:
#Post-processing the data to get the rotten tomatoes scores in the right format
for i in range(len(rt_ratings)):
    if(rt_ratings[i][-2:] == "00"):
        rt_ratings[i] = int(rt_ratings[i].strip('/100'))
    elif(rt_ratings[i][-1:] == "%"):
        rt_ratings[i] = int(rt_ratings[i].strip('%'))
print(rt_ratings[:5])



[100, 100, 97, 93, 98]


In [13]:
#Now let's take out all of the NaN values and see how much we're left with
movies_final = movies.head(1000)
movies_final.insert(0, "rt_rating", rt_ratings, False)
movies_final_1 = (movies_final[movies_final["rt_rating"] != 'NaN'])

print(len(movies_final_1))
movies_final_1.head(100)

911


,rt_rating,fn,tid,title,wordsInTitle,url,imdbRating,ratingCount,duration,year,...,Short,Sport,TalkShow,Thriller,War,Western,asdf,untitled,Unnamed: 46,Unnamed: 47
0,100,titles01/tt0012349,tt0012349,Der Vagabund und das Kind (1921),der vagabund und das kind,http://www.imdb.com/title/tt0012349/,8.4,40550,3240,1921,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
1,100,titles01/tt0015864,tt0015864,Goldrausch (1925),goldrausch,http://www.imdb.com/title/tt0015864/,8.3,45319,5700,1925,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
2,97,titles01/tt0017136,tt0017136,Metropolis (1927),metropolis,http://www.imdb.com/title/tt0017136/,8.4,81007,9180,1927,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
3,93,titles01/tt0017925,tt0017925,Der General (1926),der general,http://www.imdb.com/title/tt0017925/,8.3,37521,6420,1926,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
4,98,titles01/tt0021749,tt0021749,Lichter der Großstadt (1931),lichter der gro stadt,http://www.imdb.com/title/tt0021749/,8.7,70057,5220,1931,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,100,titles01/tt0079944,tt0079944,Stalker (1979),stalker,http://www.imdb.com/title/tt0079944/,8.2,47589,9780,1979,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
97,92,titles01/tt0080678,tt0080678,Der Elefantenmensch (1980),der elefantenmensch,http://www.imdb.com/title/tt0080678/,8.3,120156,7440,1980,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
98,94,titles01/tt0080684,tt0080684,Das Imperium schlägt zurück (1980),das imperium schl gt zur ck,http://www.imdb.com/title/tt0080684/,8.8,563252,7440,1980,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
99,94,titles01/tt0081398,tt0081398,Wie ein wilder Stier (1980),wie ein wilder stier,http://www.imdb.com/title/tt0081398/,8.3,172854,7740,1980,...,0,1,0,0,0,0,NaN,NaN,NaN,NaN


In [15]:
#Finally, let's export our result to a csv
movies_final_1.to_csv("final_data.csv", index = False, header = True)